In [4]:
from parser import parse_files
from createPopulation import createPopulation
from makespan import makespan
from avaliarPop import avaliarPop
from retornaMelhorSolucao import retornaMelhorSolucao
from selecionarPop import selecionarPop
from recombinacao import recombinacao
from mutacao import mutacao
from selecionarNovaGeracao import selecionarNovaGeracao


import time

[[2, 1, 3], [2, 3, 1], [1, 2, 3], [3, 2, 1], [2, 1, 3]]


In [5]:
POPULATION_SIZE = 10
TIME_LIMIT = 1
MUTATION_FACTOR = .3

### Ler e parsear os dados

In [6]:
files_path = ['files/tai20_5.txt', 'files/tai20_10.txt', 'files/tai20_20.txt', 'files/tai50_5.txt', 'files/tai50_10.txt', 'files/tai50_20.txt', 'files/tai100_5.txt', 'files/tai100_10.txt', 'files/tai100_20.txt', 'files/tai200_10.txt']

In [7]:
instances_list = parse_files(files_path)
# o campo processing possui as instancias a serem calculadas
print(instances_list[0])

{'processing': [[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94], [79, 3, 11, 99, 56, 70, 99, 60, 5, 56, 3, 61, 73, 75, 47, 14, 21, 86, 5, 77], [16, 89, 49, 15, 89, 45, 60, 23, 57, 64, 7, 1, 63, 41, 63, 47, 26, 75, 77, 40], [66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85, 9, 39, 41, 56, 40, 54, 77, 51, 31], [58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72, 8, 49, 47, 87, 58, 18, 68, 28]], 'number of jobs': 20, 'number of machines': 5, 'initial seed': 873654221, 'upper bound and lower bound': 1278}


### Inicializar o relatorio para cada arquivo
relatório é uma lista de dicionarios

In [8]:
## traduzir para ingles
report_list = []

print(report_list)

[]


Iniciar o laço de repetição para cada arquivo

In [9]:
from typing import List, Tuple
import itertools
import json
import random

''' Aplica o crossover entre os pais
    Vamos dividir os dois pais em três pedaços,
    Cada um desses pedaços seram recombinados
    para gerar novos filhos.
    Se os pedaços possuirem a mesma tarefa (job),
    um dos items repetidos sera substituido pelas
    tarefas restantes aleatoriamente
    Um par de pais gera n filhos 
'''
def crossover(par: Tuple[List[int], List[int]]):
  pai1, pai2 = par

  tamanho_de_cada_parte = len(pai1) // 3

  partes_pai1 = [pai1[i:i+tamanho_de_cada_parte] for i in range(0, len(pai1), tamanho_de_cada_parte)]
  partes_pai2 = [pai2[i:i+tamanho_de_cada_parte] for i in range(0, len(pai2), tamanho_de_cada_parte)]

  ## partes_pai um possui uma lista de listas, cada uma com 1/3 da tarefa
  ## ex: [[1, 2, 3], [4, 5, 6], [7, 8, 9]] para a entrada [1, 2, 3, 4, 5, 6, 7, 8, 9]
  res = []
    
  ## gera uma coninação unica entre os pedaços de cada pai
  all_combinations = list(itertools.combinations(partes_pai1 + partes_pai2, 3))
  for combination_tuple in all_combinations:
    temp = []
    for item_list in combination_tuple:
      temp += [*item_list]

    ## remove possiveis duplicatas
    ## savar como string para depois transaformar em set para eliminar possiveis repetições
    try:
      res.append(json.dumps(temp))
    except Exception as e:
      print(e)
      print(type(temp))
  unique_elements = [json.loads(x) for x in list(set(res))]

  ## como cada elemento pode possuir a mesma tarefa mais de uma vez, precisamos fazer com que esse processo seje unico
  possiveis_jobs = set(pai1)

  ## remove jobs duplicados
  novos_filhos = []

  for novo_filho in unique_elements:
    novo_filho_set = set(possiveis_jobs)
    elementos_repetidos = {}

    for job in novo_filho:
      elementos_repetidos[job] = 1 + elementos_repetidos.get(job, 0)

      if job in novo_filho_set:
        novo_filho_set.remove(job)

    for job_index in range(len(novo_filho)):
      if elementos_repetidos[novo_filho[job_index]] > 1:
        elementos_repetidos[novo_filho[job_index]] -= 1
        novo_filho[job_index] = random.choice(list(novo_filho_set))
        novo_filho_set.remove(novo_filho[job_index])

    novos_filhos.append(novo_filho)
  
  ## retira possiveis filhos duplicados
  json_item = [json.dumps(x) for x in novos_filhos]

  json_list_set = list(set(json_item))

  filhos_unicos = [json.loads(x) for x in json_list_set]
  
  return filhos_unicos

def recombinacao(populacaoSelecionada: List[Tuple[List[int], List[int]]]):
  return [crossover(par) for par in populacaoSelecionada]


In [10]:
for instance_index in range(1):
    # Começa vazio em todos os casos
    melhoresSolucoes = []
    todasAsMelhoresSolucoes = []

    # Para cada instância executar todo o algoritmo 10 vezes

    for i in range(10):
        current_instance = instances_list[instance_index]
        melhorSolucao = {'solucao':[], 'aptidao': float('inf'), 'tempoFinal':0}
        startTime = time.time()

        number_of_jobs = current_instance['number of jobs']
        instancia = current_instance['processing']

        populacao = createPopulation(number_of_jobs, POPULATION_SIZE)
        geracao = 0

        while True:
            geracao += 1
            if geracao % 1000 == 0:
                print(TIME_LIMIT)
                print(time.time() - startTime)
                print(TIME_LIMIT <= time.time() - startTime)
            if TIME_LIMIT <= time.time() - startTime:
                break

            # criar segundo criterio de parada
            criterioParada2 = False

            if criterioParada2 == True:
                break

            aptidaoPop = avaliarPop(populacao, instancia)
            melhorSolucaoAtual = retornaMelhorSolucao(populacao, aptidaoPop)

            todasAsMelhoresSolucoes.append({**melhorSolucaoAtual, "tempoFinal": time.time() - startTime, "geração": geracao})

            if melhorSolucao['aptidao'] > melhorSolucaoAtual['aptidao']:
                melhorSolucao = melhorSolucaoAtual

            populacaoSelecionada = selecionarPop(
                populacao, aptidaoPop, POPULATION_SIZE)

            novasSolucoes = recombinacao(populacaoSelecionada)
            novasSolucoes = mutacao(novasSolucoes, MUTATION_FACTOR)

            selecionarNovaGeracao(populacaoSelecionada,
                                  novasSolucoes, POPULATION_SIZE)
        pass

        melhorSolucao['tempoFinal'] = time.time() - startTime
        melhoresSolucoes.append(melhorSolucao)
        print(melhorSolucao)

    report_list.append({'melhor': melhorSolucao, 'todasAsMelhores': melhoresSolucoes, 'todasAsTrocas': todasAsMelhoresSolucoes})


{'aptidao': 1452, 'solucao': [16, 12, 3, 19, 14, 11, 9, 13, 17, 18, 10, 8, 5, 7, 1, 2, 15, 20, 6, 4], 'tempoFinal': 1.0005760192871094}
{'aptidao': 1465, 'solucao': [11, 7, 6, 10, 9, 2, 8, 5, 13, 14, 15, 19, 12, 3, 20, 4, 17, 18, 16, 1], 'tempoFinal': 1.0050959587097168}
{'aptidao': 1414, 'solucao': [3, 4, 11, 15, 1, 9, 6, 8, 12, 17, 13, 10, 14, 20, 19, 16, 18, 5, 2, 7], 'tempoFinal': 1.0017585754394531}
{'aptidao': 1420, 'solucao': [13, 14, 1, 7, 11, 3, 4, 9, 2, 8, 20, 19, 12, 15, 18, 17, 6, 16, 5, 10], 'tempoFinal': 1.002098798751831}
{'aptidao': 1438, 'solucao': [5, 17, 20, 14, 6, 7, 11, 13, 8, 9, 2, 1, 16, 15, 4, 18, 19, 10, 12, 3], 'tempoFinal': 1.0024693012237549}
{'aptidao': 1397, 'solucao': [14, 9, 13, 15, 6, 19, 1, 3, 11, 8, 12, 4, 18, 7, 16, 20, 17, 2, 10, 5], 'tempoFinal': 1.0004889965057373}
{'aptidao': 1425, 'solucao': [8, 14, 3, 11, 5, 6, 20, 15, 12, 19, 13, 7, 4, 9, 2, 17, 18, 16, 10, 1], 'tempoFinal': 1.000810146331787}
{'aptidao': 1424, 'solucao': [3, 2, 19, 9, 6, 5, 1

Salvar relatório

In [37]:
import numpy as np
import csv

# salvarRelatorio(relatorio)
instancia_resultado = report_list[0]

lower_bound = instancia_resultado['melhor']['aptidao']

### encontrar upperBound
upper_bound = lower_bound

lista_dos_melhores_tempos = []
lista_das_melhores_aptidoes = []

for solucao in instancia_resultado['todasAsMelhores']:
  upper_bound = max(solucao['aptidao'], upper_bound)
  lista_dos_melhores_tempos.append(solucao['tempoFinal'])
  lista_das_melhores_aptidoes.append(solucao['aptidao'])


desvio_padrao_tempo = np.std(lista_dos_melhores_tempos)
desvio_padrao_aptidao = np.std(lista_das_melhores_aptidoes)

variancia_tempo = np.var(lista_dos_melhores_tempos)
variancia_aptidao = np.var(lista_das_melhores_aptidoes)

media_tempo = np.mean(lista_dos_melhores_tempos)
media_aptidao = np.mean(lista_das_melhores_aptidoes)

mediana_tempo = np.median(lista_dos_melhores_tempos)
mediana_aptidao = np.median(lista_das_melhores_aptidoes)

items = [desvio_padrao_tempo, desvio_padrao_aptidao, variancia_tempo, variancia_aptidao, media_tempo, media_aptidao, mediana_tempo, mediana_aptidao, lower_bound, upper_bound]
nomes = ['desvio_padrao_tempo', 'desvio_padrao_aptidao', 'variancia_tempo', 'variancia_aptidao', 'media_tempo', 'media_aptidao', 'mediana_tempo', 'mediana_aptidao', 'lower_bound', 'upper_bound']
dic = dict(zip(nomes, items))

with open('mycsvfile.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, dic.keys())
    w.writeheader()
    w.writerow(dic)


# lower bound, upper bound, valores médios e desvios (tanto para aptidão quanto para o tempo de execução)

TypeError: DictWriter.writerow() takes 2 positional arguments but 3 were given